In [2]:
# import stuff
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import math
import os
import timeit

In [23]:
# load CSV file
def load_csv(filename):
  dataset = []

  with open(filename, 'r') as file:
    next(file) # skip the first line ( the 'header' )
    csv_readder = csv.reader(file)
    
    for row in csv_readder:
      if not row:
        continue
      
      dataset.append(row)

  return dataset


# clear values in 'dictionary'
def clear_values_in_dic(dic1):
  dic1 = dic1.fromkeys(dic1, 0)
  # print('dic1.items() >>> ' , dic1.items() )

  return dic1  


# make a 'dictionary' from sentences
def convert_to_dic(dataset):
  dic1 = {}; # empty dictionary

  for row in dataset:
    # print(row[0].lower().split(" ") ) # row, 1st item, the sentence
    # print(row[1].lower().split(" ") ) # row, 2nd item, the 'answer'
    
    word_list = row[0].lower().split(" ")
    for word in word_list:
      # print(word)
      dic1[word] = 0

  return dic1


# convert from 'dictionary' to 'vector'
## input: [ ['I buy an apple phone', 'happy'], ['I eat the gig apple', 'happy'] ]
## 2 sentences go in, 2 vectors come out
def convert_sentences_to_vector(dic1, sentence_list):
  vector_list = []

  for idx, row in enumerate(sentence_list):
    dic1 = clear_values_in_dic(dic1)

    sentence = row[0].lower().split()
    answer = row[1].lower()
    # print(sentence)
    # print(answer)

    for word in sentence:
      # print(word)
      for key, value in dic1.items():
        # print(key)
        if (word == key):
          dic1[key] = 1
      
    # print( list( dic1.values() ) )
    vector_list.append( [list(dic1.values() ), answer ] )

  return vector_list



def predict_answer( test_row, train_dataset, k_nay ):
  my_naybers = find_naybers(test_row, train_dataset, k_nay)
  
  my_naybers_answer = [ row[-1] for row in my_naybers ] # >> [' ', 'happy', 'happy'], >> row[-1] the last 'column' in a row
  # print( my_naybers_answer )
  the_prediction = max( set(my_naybers_answer), key=my_naybers_answer.count ) # # set() math 'set' you don't have the same stuff in a 'set'
  # print( the_prediction )

  return the_prediction



def find_naybers(test_row, train_data, k_nay):
  distance_list = []
  man_distance = 0.0

  for train_row in train_data:
  
    # check if stuff in 'test_row' is the same as 'train_row'
    if ( test_row[0] == train_row[0] ):
      print(f"*itself - itself")
      print(f"*skipped test_row[0]: {test_row[0]}  train_row[0]: {train_row[0]} \n")    
      pass

    else:
      man_distance = calculate_manhat_distance(test_row, train_row)
      # print(f"train_row: {train_row}")
      # print(f"train_row[1]: {train_row[1]}")
      distance_list.append( (man_distance, train_row[1]) )
    

  print( distance_list )
  distance_list.sort( key=lambda asdf: asdf[0] ) # asdf[0] is just the 'first item (man distance)' in the 'distance_list'
  print( distance_list )

  naybers = []
  for i in range(k_nay):
    naybers.append( distance_list[i] )

  return naybers


def calculate_manhat_distance(test_row, train_row):
  distance = 0.0

  # print(f"test_row: {test_row}  train_row: {train_row}")  
  # print(f"test_row[0]: {test_row[0]}  train_row[0]: {train_row[0]}")  
  if (  len(test_row[0]) == len(train_row[0])   ):
    print(f"test_row[0]: {test_row[0]}  train_row[0]: {train_row[0]}")  
      
    for a, b in zip( test_row[0], train_row[0]  ):
    # print('calculate_manhat_distance a: ', a)
    # print('calculate_manhat_distance b: ', b)
      distance = distance + abs(a-b)      
    
  else:
    print('ERROR, vector length do not match')
    
  return distance


# v1 = [[2,3],'yes']
# v2 = [[5,7],'no']
# v2 = [[5,8],'no'] # uncommit for error checking

# dis1 = calculate_manhat_distance(v1, v2)
# print(f"checking v1 - v2 should be 7, got : {dis1}")

#if condition returns False, AssertionError is raised:
# assert dis1 == 7, "v1 - v2 should be 7"



def calculate_eu_distance(test_row, train_row):
  distance = 0.0

  # print(f"test_row: {test_row}  train_row: {train_row}")  

  if ( len(test_row[0]) == len(train_row[0]) ):

    for a, b in zip( test_row[0], train_row[0] ):
      # print('calculate_eu_distance a: ', a)
      # print('calculate_eu_distance b: ', b)

      distance = distance + (a - b)**2
  else:
    print('ERROR, vector length do not match')
    # pass

  return math.sqrt( distance )



def accuracy_met(actual, predicted):
  correct = 0

  # print(f"actual {actual}")
  # print(f"predicted {predicted}")

  for idx in range( len(actual) ): # >> idx 0,1,2
    if actual[idx] == predicted[idx]:
      correct = correct + 1
  
  # print(f"correct: {correct}  out of total: { len(actual) } "  )
  # print( correct / float( len(actual) ) * 100 )
  return correct / float( len(actual) ) * 100
    


def k_eval(test_set, train_set, actual): 
  k_evaluation = []

  for k in range(1, 22, 3):
    preds = []

    for row in test_set:
      predictors_only = row[ : -1]
      prediction = predict_answer(predictors_only, train_set, k)

      preds.append( prediction ) 
    
    curr_accuracy = accuracy_met(actual, preds)
    k_evaluation.append( (k, curr_accuracy) )

  return k_evaluation



In [17]:
## this block of code below is for running 
## train_set.csv, validation_set.csv
## loading data
## why? to pick the value of 'k'

# getting file
filename_train_set_simple = '/content/drive/MyDrive/homework1 spring 2021/data/train_set_simple.csv'
print(f"filename: {filename_train_set_simple}")

# load csv file into a variable
train_set_simple = load_csv(filename_train_set_simple)
print(f"current train_set: {train_set_simple}")

# creating 'dict/ionary' from train_set.csv
train_set_dic = convert_to_dic(train_set_simple)
# print(f"train_set_dic.items() {train_set_dic.items()} "  )
# print(f"len( train_set_dic.items() ) {len( train_set_dic.items() )}")

# convert sentences into vectors
train_set_simple_vec = convert_sentences_to_vector(train_set_dic, train_set_simple)
# len( train_set_simple_vec[0][0] ) # >> should be 
# train_set_simple_vec[0][0] # >> should be the 'emotions'
# train_set_simple_vec[0][1] # >> should be the 'emotions'

actual_emotions = np.array(train_set_simple)[ : , -1] # >> use np.array() to get the 'emotions' from each sentence
# len( actual_emotions )
# print(f"actual_emotions {actual_emotions}")


filename: /content/drive/MyDrive/homework1 spring 2021/data/train_set_simple.csv
current train_set: [['europe retain trophy with big win', 'joy'], ['senate votes to revoke pensions', 'sad'], ['the amounts you have to pay for a bomb scare', 'fear'], ['pair of satellites will document sun in d', 'joy'], ['malaysian airasia x to fly in july', 'joy'], ['dow hits new record eyes', 'joy'], ['bathing mom awakes to find baby dead', 'sad'], ['we re a pretty kind bully', 'joy'], ['women in their s are perfectly good mothers', 'sad']]


In [24]:
## debugging block
## skipping calcualtion on the same row
# how to check if content inside 'list' are the same

## debug code
train_set111 = [ 
             [[ 1, 1, 1, 1, 1], 'happy'], 
              [[1, 0, 0, 1, 0], 'happy'],
              [[0, 0, 0, 1, 0], 'sad'],
             [[1, 1, 0, 1, 1], 'happy'] 
             ]

# train_set[1][0] == train_set[2][0]

for idx, row in enumerate(train_set111):
  
  test_row = row
  print(f"test_row {test_row}")
  # print(f"row index: {idx}")
  

  prediction = predict_answer(test_row, train_set111, 3)
  print(f"prediction: {prediction} \n")


test_row [[1, 1, 1, 1, 1], 'happy']
*itself - itself
*skipped test_row[0]: [1, 1, 1, 1, 1]  train_row[0]: [1, 1, 1, 1, 1] 

test_row[0]: [1, 1, 1, 1, 1]  train_row[0]: [1, 0, 0, 1, 0]
test_row[0]: [1, 1, 1, 1, 1]  train_row[0]: [0, 0, 0, 1, 0]
test_row[0]: [1, 1, 1, 1, 1]  train_row[0]: [1, 1, 0, 1, 1]
[(3.0, 'happy'), (4.0, 'sad'), (1.0, 'happy')]
[(1.0, 'happy'), (3.0, 'happy'), (4.0, 'sad')]
prediction: happy 

test_row [[1, 0, 0, 1, 0], 'happy']
test_row[0]: [1, 0, 0, 1, 0]  train_row[0]: [1, 1, 1, 1, 1]
*itself - itself
*skipped test_row[0]: [1, 0, 0, 1, 0]  train_row[0]: [1, 0, 0, 1, 0] 

test_row[0]: [1, 0, 0, 1, 0]  train_row[0]: [0, 0, 0, 1, 0]
test_row[0]: [1, 0, 0, 1, 0]  train_row[0]: [1, 1, 0, 1, 1]
[(3.0, 'happy'), (1.0, 'sad'), (2.0, 'happy')]
[(1.0, 'sad'), (2.0, 'happy'), (3.0, 'happy')]
prediction: happy 

test_row [[0, 0, 0, 1, 0], 'sad']
test_row[0]: [0, 0, 0, 1, 0]  train_row[0]: [1, 1, 1, 1, 1]
test_row[0]: [0, 0, 0, 1, 0]  train_row[0]: [1, 0, 0, 1, 0]
*itself - 

In [ ]:
## this block of code below is for running train_set.csv, validation_set.csv
## why? to pick the value of 'k'

train_set = train_set_simple_vec
actual = np.array(train_set)[ : , -1]
print(f"actual {actual}")

# k_list = [3, 5, 7, 9, 11, 13, 15,17 ]
k_list = [3,5,7]

for k in k_list:
  preds3 = []

  print(f"\nlen train_set: { len(train_set) } ")
  print(f"len actual: { len(actual) } \n")
  start = timeit.default_timer() # start program timer

  for idx, row in enumerate(train_set): 
    test_row = row
    the_prediction = predict_answer(test_row, train_set, k)
    preds3.append( the_prediction )

  cur_accuracy = accuracy_met(actual, preds3)
  stop = timeit.default_timer() # stop program timer

  print(f"when K is {k}, accuracy %: {cur_accuracy}")
  print(f"program run time (seconds):  {stop - start} ")  

actual ['joy' 'sad' 'fear' 'joy' 'joy' 'joy' 'sad' 'joy' 'sad']

len train_set: 9 
len actual: 9 

*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
when K is 3, accuracy %: 77.77777777777779
program run time (seconds):  0.0013607219998448272 

len train_set: 9 
len actual: 9 

*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
when K is 5, accuracy %: 55.55555555555556
program run time (seconds):  0.001663826999902085 

len train_set: 9 
len actual: 9 

*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
*itself - itself
when K is 7, accuracy %: 55.55555555555556
program run time (seconds):  0.0010455629999341909 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [ ]:
## this block of code is for test_set.csv
## loading all the csv files
## final step, predicting answers


file_name_train_set = '/content/drive/MyDrive/homework1 spring 2021/data/train_set.csv'
# file_name = '/content/drive/MyDrive/homework1 spring 2021/data/validation_set.csv'
file_test_set_no_textid = '/content/drive/MyDrive/homework1 spring 2021/data/test_set_no_textid.csv'
file_name_for_making_dic = '/content/drive/MyDrive/homework1 spring 2021/data/train_vali_test_for_dic.csv'

dataset_to_make_dic = load_csv(file_name_for_making_dic)
# print(f"dataset1 {dataset1}")

# create 'dictionary' of words here
dic1 = convert_to_dic(dataset_to_make_dic)

print(f"file_name: {file_name_for_making_dic}")
print(f"dic1.items() {dic1.items()} "  )
print(f"len( dic1.items() ) {len( dic1.items() )}") 

file_name: /content/drive/MyDrive/homework1 spring 2021/data/train_vali_test_for_dic.csv
dic1.items() dict_items([('europe', 0), ('retain', 0), ('trophy', 0), ('with', 0), ('big', 0), ('win', 0), ('senate', 0), ('votes', 0), ('to', 0), ('revoke', 0), ('pensions', 0), ('the', 0), ('amounts', 0), ('you', 0), ('have', 0), ('pay', 0), ('for', 0), ('a', 0), ('bomb', 0), ('scare', 0), ('pair', 0), ('of', 0), ('satellites', 0), ('will', 0), ('document', 0), ('sun', 0), ('in', 0), ('d', 0), ('malaysian', 0), ('airasia', 0), ('x', 0), ('fly', 0), ('july', 0), ('dow', 0), ('hits', 0), ('new', 0), ('record', 0), ('eyes', 0), ('bathing', 0), ('mom', 0), ('awakes', 0), ('find', 0), ('baby', 0), ('dead', 0), ('we', 0), ('re', 0), ('pretty', 0), ('kind', 0), ('bully', 0), ('women', 0), ('their', 0), ('s', 0), ('are', 0), ('perfectly', 0), ('good', 0), ('mothers', 0), ('hands', 0), ('on', 0), ('doomsday', 0), ('clock', 0), ('move', 0), ('forward', 0), ('italy', 0), ('hold', 0), ('no', 0), ('show', 0),

In [ ]:
## this block of code is for test_set.csv
## loading all the csv files
## final step, predicting answers


# open train_set.csv file
train_set_file = load_csv(file_name_train_set)
# display(train_set_file)

# convert train_set.csv to vector, using dic1
train_set = convert_sentences_to_vector(dic1, train_set_file)
# len( train_set[0][0] ) >> should be 3,275
# train_set[0][1] >> should be the 'emotions'

# open test_set.csv file
test_set_file = load_csv(file_test_set_no_textid)
# display(test_set_file)

# convert test_set.csv to vector, using dic1
test_set = convert_sentences_to_vector(dic1, test_set_file)
# len( test_set[0][0] ) # >> should be 3,275
# test_set[0][1] # >>  should be the '' nothing


In [ ]:

## this block of code is for test_set.csv
## the 'k' value will be 3
k_best = 3

print(f"len train_set: { len(train_set) } ")
print(f"len test_set: { len(test_set) } \n")


pred_final = []
start = timeit.default_timer() # start program timer


for idx, test_row in enumerate(test_set):
  the_prediction = predict_answer(test_row, train_set, k_best)
  pred_final.append( [the_prediction, idx] )  


stop = timeit.default_timer() # stop program timer
print(f"pred_final {pred_final}\n")
print(f"program run time (seconds):  {stop - start} ")  

len train_set: 598 
len test_set: 312 

pred_final [['sad', 0], ['anger', 1], ['sad', 2], ['joy', 3], ['joy', 4], ['fear', 5], ['surprise', 6], ['surprise', 7], ['surprise', 8], ['joy', 9], ['sad', 10], ['surprise', 11], ['fear', 12], ['sad', 13], ['fear', 14], ['fear', 15], ['anger', 16], ['anger', 17], ['sad', 18], ['joy', 19], ['sad', 20], ['sad', 21], ['sad', 22], ['surprise', 23], ['joy', 24], ['anger', 25], ['sad', 26], ['surprise', 27], ['anger', 28], ['joy', 29], ['surprise', 30], ['fear', 31], ['sad', 32], ['anger', 33], ['joy', 34], ['joy', 35], ['surprise', 36], ['fear', 37], ['anger', 38], ['surprise', 39], ['joy', 40], ['fear', 41], ['joy', 42], ['disgust', 43], ['anger', 44], ['joy', 45], ['joy', 46], ['surprise', 47], ['sad', 48], ['sad', 49], ['sad', 50], ['sad', 51], ['surprise', 52], ['joy', 53], ['sad', 54], ['joy', 55], ['sad', 56], ['sad', 57], ['joy', 58], ['sad', 59], ['sad', 60], ['joy', 61], ['anger', 62], ['surprise', 63], ['joy', 64], ['sad', 65], ['joy', 66]

In [ ]:
## this block of code is for
## writing the pred_final 'list' 
## into a csv file

dat1 = [ [ [1,2,3], 'yes'], [[4,5,6],'no'] ]

# open a csv file in 'w' mode
with open('111.csv', 'w', newline='') as file:
  write1 = csv.writer(file)
  write1.writerow( ['emotions', 'txtid'] )

  for item in pred_final:
    write1.writerow(item)
  


In [ ]:
## backup code
##

k_start = 3
k_stop = math.floor ( math.sqrt( len(actual) ) )
print(f"k_start: { k_start } to k_stop: {k_stop} \n")
k_eval1 = []

for k in range(k_start, k_stop, 2): # 1 to 9, increment by 2 >> 1,3,5,7,9
  preds2 = []

  for row in train_set:
    the_prediction = predict_answer(row, train_set, k) # >> 'row' is the current row you use as a 'test_row'
    preds2.append( the_prediction )
  
  current_accuracy = accuracy_met(actual, preds2)
  k_eval1.append( (k, current_accuracy) )


print(f"number of K, accuracy: {k_eval1}")

k_start: 3 to k_stop: 3 

number of K, accuracy: []


In [ ]:
## backup code
##

# test_row = [['My friend has an apple', '']]
# test_row_vector = convert_sentences_to_vector(dic1, test_row)
# print(test_row_vector)

# test_set = [ [[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], ''] ]
# test_row = [[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], ''] 

# train_row1 = [[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'happy']
# train_row2 = [[1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 'happy']
# train_row3 = [[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0], 'sad']

# train_set = [ [[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'happy'], 
#               [[1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 'happy'],
#               [[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0], 'sad'] ]
# calculate_eu_distance(test_row, train_row3)


# predict1 = predict_answer(test_row, train_dataset, 3)
# actual = ['happy', 'happy', 'sad']
# predicted = ['sad', 'happy', 'sad']